## Required Imports

In [92]:
import h5py
import matplotlib.pyplot as plt
import json
import numpy as np
import plotly.graph_objects as go
import lmfit as lm

from common import save_to_igor_itx, calculate_fit, i_sense, entropy_weak, get_data_index, bin_data

from squarewave import process_per_row_parts, SquareProcessedData, process_avg_parts, get_data_part, single_wave_masks, convert_to_4_setpoint_AW

## Data -- Square Wave Parts

In [93]:
datnum = 2164
with h5py.File(f'data/dat{datnum}.h5', 'r') as f:
    data = f['cscurrent_2d'][:]
    data_x = f['x_array'][:]
    sweeplogs = f['metadata'].attrs['sweep_logs']
    aw = f['fdAW_0'][:]
    data_y = f['y_array'][:]
sweeplogs = json.loads(sweeplogs)
awg_info = sweeplogs['FastDAC 1']['AWG']
num_pts = awg_info['waveLen']
num_steps = awg_info['numSteps']
duration = num_pts/awg_info['measureFreq']
measure_freq = sweeplogs['FastDAC 1']['MeasureFreq']
x = np.linspace(0, duration, num_pts)
data_x = np.linspace(data_x[0], data_x[-1], num_steps)

aw = convert_to_4_setpoint_AW(aw)
data = np.mean(data, axis=0)  # From 2D to 1D transition data

In [94]:
sweeplogs

{'comment': 'null',
 'filenum': 2164,
 'axis_labels': {'x': 'ACC/100 (mV)', 'y': 'Repeats'},
 'current_config': 'sc1615787252.json',
 'time_completed': 'March 14, 2021 23:58:32',
 'time_elapsed': 4149,
 'FastDAC 1': {'visa_address': 'ASRL4::INSTR',
  'SamplingFreq': 2538.1,
  'MeasureFreq': 2538.1,
  'DAC0{csbias/100}': -31.128,
  'DAC1{ACC/100}': 199.9,
  'DAC2{CSQ}': -188.6,
  'DAC3{ESS}': -350.65,
  'DAC4{ESC}': -349.43,
  'DAC5{ESP}': -403.14,
  'DAC6{HO1/10M}': 0,
  'DAC7{HO2/1000}': 0,
  'ADC0': 704.23,
  'ADC1': 3.1419,
  'ADC2': 4.1079,
  'ADC3': 1.282,
  'AWG': {'AW_Waves': '0,1',
   'AW_Dacs': '6,7',
   'waveLen': 204,
   'numADCs': 1,
   'samplingFreq': 2538.1,
   'measureFreq': 2538.1,
   'numWaves': 2,
   'numCycles': 1,
   'numSteps': 498}},
 'Lakeshore': {'Temperature': {'MC K': 0.10002,
   '50K Plate K': 58.388,
   '4K Plate K': 3.2414,
   'Magnet K': 3.838,
   'Still K': 0.90953}},
 'BabyDAC': {'DAC0{CSS}': -145,
  'DAC1{IS}': -150,
  'DAC2{BD2D}': -400,
  'DAC3{BD2S}'

In [95]:
bin_size = 1  # Binning after processing before plotting

x = bin_data(x, bin_x=bin_size)

In [96]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_x, y=data[::num_pts], mode='lines'))
fig.show()

#### Average data for different parts of square wave applied

In [97]:
# peak = -66  # peak position in mV
# dip = 110   # dip position in mV
# width = 80  # + - this value averaged around the peak/dip in mV
# peak = -73  # peak position in mV
peak = -13  # peak position in mV
dip = 0.4   # dip position in mV
width = 5  # + - this value averaged around the peak/dip in mV

widx = round(width/np.mean(np.diff(data_x)))
peak_idx, dip_idx = get_data_index(data_x, [peak, dip], is_sorted=True)
regions = np.s_[0:widx*2], np.s_[peak_idx-widx:peak_idx+widx], np.s_[dip_idx-widx: dip_idx+widx], np.s_[-widx*2:-1]

all_avg = np.reshape(data, (-1, num_pts))  # Line up all square wave cycles on top of each other
region_names = ('zeroL', 'pos', 'neg', 'zeroR')
masks = single_wave_masks(aw)

In [98]:
import plotly.graph_objects as go
all_avg.shape
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_x, y=np.mean(all_avg, axis=1)))
fig.show()

In [99]:
data_x.shape, all_avg.shape

((498,), (498, 204))

In [100]:
avgs = list()
setpoint_averaging_delay_ms = 3  # How many ms to throw out before averaging setpoint values
setpoint_start_index = round(setpoint_averaging_delay_ms/1000*measure_freq)
print(f'Setpoint start index = {setpoint_start_index}')

for region, name in zip(regions, region_names):
    print(region)
    # Checking what data is being averaged
    # fig = go.Figure(layout=dict(title=dict(text=f'{region}')))
    # fig.add_trace(go.Scatter(x=data_x[region], y=np.mean(all_avg[region], axis=1)))
    # fig.show()
    ################

    avg = np.mean(all_avg[region], axis=0)  # Average all square waves in region together to make one averaged square wave
    avg = avg - np.mean([np.nanmean((avg*masks[0])[setpoint_start_index:]), np.nanmean((avg*masks[2])[setpoint_start_index:])])  # Subtract mean value of 0 heating parts
    avgs.append(avg)


# Separate single averaged square wave into the individual parts
datas = dict()
for avg, name in zip(avgs, region_names):
    datas[name] = list()
    for mask in masks:
        data = avg*mask
        data = bin_data(data, bin_x=bin_size)
        datas[name].append(data) 
        
        

Setpoint start index = 8
slice(0, 12, None)
slice(226, 238, None)
slice(243, 255, None)
slice(-12, -1, None)


## Quick plot before exporting data to Igor for final Figure

In [101]:
x.shape, d.shape

((204,), (204,))

In [102]:
for name, data in datas.items():
    fig = go.Figure()

    fig.update_layout(xaxis_title='Time through Square Wave /s', yaxis_title=f'Current /nA',
                                  title=f'Data averaged to one Square Wave')

    for d, label in zip(data, ['00', '+', '01', '-']):
        fig.add_trace(go.Scatter(y=d, x=x, mode='lines', name=label))
    fig.show()

## Save to .itx file for final Figure in Igor

In [103]:
np.array(list(datas.values())).reshape((16, -1)).shape

(16, 204)

In [104]:
# save_to_igor_itx(f"exports/SWData.itx",
#                  xs=[x]*4*4,
#                  datas=list(np.array(list(datas.values())).reshape((4*4, -1))),
#                  names=[f'SWData_{name}_{label}' for name in datas.keys() for label in ['v0_0', 'vP', 'v0_1', 'vM']])